<a href="https://colab.research.google.com/github/Bryant-Dental/raptor_functions/blob/main/raptor_functions/examples/supervised_end_to_end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Libraries

In [ ]:
# !pip install raptor_functions

In [4]:
# !pip install mlflow optuna Boruta pycaret awscli boto3 tsfresh

### Configure AWS
- This is necessary to use train_experiments to log results and artifacts on aws instance

In [4]:
# !aws configure

### Import Packages

In [5]:
import pandas as pd

In [8]:
PACKAGE_FOLDER = '/raptor/raptor_functions'

print(PACKAGE_FOLDER)
# sys.path.insert(0, PACKAGE_FOLDER)
sys.path.append(PACKAGE_FOLDER)

/raptor/raptor_functions


In [9]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import mlflow
from raptor_functions.supervised.prediction import load_model, make_prediction, get_model_and_features, get_prediction_features
import xgboost as xgb
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from raptor_functions.supervised.train import train_experiments
from raptor_functions.supervised.datasets import get_data
from raptor_functions.supervised.feature_extraction import get_training_features




### Load Data

In [10]:
df = get_data('handheld_data')
df.head()

,exp_unique_id,exp_name,timesteps,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10,sensor_11,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,Humidity (r.h.),measurement_stage,date_exp,time_elapsed,datetime_exp_start,datetime_exp,filename,result
0,0,Test 24,1,131.938,132.443,177.444,178.157,44.974,44.508,219.374,71.286,159.292,21.965,162.449,36.144,52.439,110.970,124.839,138.676,60.717,273.972,26.905,25.456,189.092,188.708,143.288,142.050,46.07,baseline,21/3/2022,0.00,14:12:54,2022-03-21 14:12:54.000,5 NegTest_3_20220321_14_12.txt,Control
1,0,Test 24,2,133.078,133.972,177.444,178.694,45.131,44.611,219.607,71.927,161.653,21.927,162.290,36.144,52.211,111.181,124.364,141.232,61.096,274.289,27.067,25.536,189.285,188.516,142.874,141.914,46.07,baseline,21/3/2022,0.25,14:12:54,2022-03-21 14:12:54.250,5 NegTest_3_20220321_14_12.txt,Control
2,0,Test 24,3,131.812,132.064,177.622,178.694,45.183,44.663,219.841,72.070,162.130,21.965,162.290,36.190,52.268,111.075,124.364,141.641,61.159,274.289,27.027,25.496,189.285,188.516,143.288,141.914,46.07,baseline,21/3/2022,0.50,14:12:54,2022-03-21 14:12:54.500,5 NegTest_3_20220321_14_12.txt,Control
3,0,Test 24,4,131.059,132.064,177.444,178.694,45.183,44.663,219.607,72.070,161.812,21.927,161.971,36.237,52.211,111.075,124.720,141.914,61.159,274.289,27.067,25.496,189.092,188.516,142.599,141.368,46.07,baseline,21/3/2022,0.75,14:12:54,2022-03-21 14:12:54.750,5 NegTest_3_20220321_14_12.txt,Control
4,0,Test 24,5,131.435,131.938,177.622,178.694,45.131,44.663,219.607,72.286,161.971,22.003,161.971,36.144,52.268,111.075,125.673,142.187,61.286,273.972,26.986,25.456,189.092,188.516,142.187,140.825,46.07,baseline,21/3/2022,1.00,14:12:54,2022-03-21 14:12:55.000,5 NegTest_3_20220321_14_12.txt,Control


### Feature Engineering
- Features extracted from each sensor array  for all cycles of experiment. 
- These include piecemeal features like mean, median, std, variance and transform features eg FFT and wavelength transform. 
- There is option to use only the raw signals or add the offset and gradient signals 
- Boruta algorithm is used to select relevant features
- Default model for boruta is xgboost but currently does not work in google colab. 
- It is advisable to use another tree model e.g random forest

In [11]:

forest = RandomForestClassifier()

In [12]:
df = get_training_features(df, offset=True, gradient=True, tree_model=forest)

Extracting all features


Feature Extraction: 100%|██████████| 10/10 [21:53<00:00, 131.35s/it]


Selecting relevant features


Error: Canceled future for execute_request message before replies were done

In [10]:
df.head()

,"sensor_5__fft_coefficient__attr_""angle""__coeff_34","sensor_23__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""max""","sensor_5offset__fft_coefficient__attr_""angle""__coeff_34","sensor_5gradient__fft_coefficient__attr_""imag""__coeff_34","sensor_5gradient__fft_coefficient__attr_""angle""__coeff_34",sensor_22gradient__number_cwt_peaks__n_1,result
0,27.346596,161.547000,27.346596,0.257730,40.181126,15.0,Control
1,123.682517,160.612167,123.682517,0.144977,136.710298,16.0,Covid
2,-23.344104,164.514500,-23.344104,0.183914,14.824050,15.0,Control
3,-156.476693,161.026167,-156.476693,-0.262346,-61.980664,11.0,Covid
4,-13.985148,163.417667,-13.985148,0.305857,18.978355,12.0,Control


### Model Training

In [11]:
train_experiments(df)

[W 2022-04-22 08:15:35,446] Trial 4 failed, because the value None could not be cast to float.


Click on this link to track experiments:  http://ec2-3-10-175-206.eu-west-2.compute.amazonaws.com:5000/


### Load Model

In [ ]:

model_uri = 's3://raptor-mlflow-data/mlartifacts/1/f7f804f6f0594f2dbc8e03cb83fde6e9/artifacts/Extreme Gradient Boosting'
loaded_model = load_model(model_uri)


In [13]:
# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Load model as a Sklearn flavor.
loaded_model = mlflow.sklearn.load_model(model_uri)



In [ ]:
# Load model from pickle file.
model_pickle_filepath  = 'path/to/pickle_model'

loaded_model = load_model(model_uri)

## Prediction

In [ ]:
from raptor_functions.supervised.datasets import get_data
df = get_data('handheld_data')

# select one single experiment
df_exp = df.groupby('exp_unique_id').get_group(0)

In [14]:
X = df_exp.drop('result', axis=1)

### Method 1

In [ ]:
prediction_data = get_prediction_features(X, model_uri)

In [ ]:
prediction = loaded_model.predict(prediction_data)

### Method 2

In [ ]:
pred = make_prediction(X, model_uri)